# Aeroacoustics

In this notebook we showcase how to set up and run an aeroacoustic simulation for a quadcopter using the Flow360 Python API. The process involves creating a project, defining multiple simulation runs by forking cases, and retrieving aeroacoustic results.

## 1. Setup and Imports

We begin by importing libraries and modules. `flow360` is the main package, and `pandas` is used for data handling of the results.

In [1]:
import pandas as pd

import flow360 as fl
from flow360.examples import Quadcopter

## 2. Project Creation

A Flow360 `Project` is a container for simulation assets such as geometries, meshes and cases. Here, the project is initialized from a pre-existing volume mesh file.

In [2]:
Quadcopter.get_files()

project = fl.Project.from_volume_mesh(
    Quadcopter.mesh_filename, name="Aeroacoustic results from Python"
)
vm = project.volume_mesh

[12:38:50] INFO: VolumeMesh successfully submitted:                                                                
                   type   = Volume Mesh                                                                            
                   name   = Aeroacoustic results from Python                                                       
                   id     = vm-a4b4e3cf-ef87-434b-9244-ef9753752cf1                                                
                   status = uploaded                                                                               
           

## 3. Simulation Parameters

This section defines the simulation parameters. This includes defining the rotating zones, unsteady time-stepping parameters, and the physical models for the simulation.


### Rotation Zones

Provided volume mesh conatins four separate rotating zones, one for each rotor. These zones need to have their center of rotation and axis specified. The rotational speed `omega` is also defined.

In [3]:
with fl.SI_unit_system:
    rotation_zone_1 = vm["zone_r1"]
    rotation_zone_1.center = [-0.125, 0.125, 0.0055]
    rotation_zone_1.axis = [0, 0, 1]

    rotation_zone_2 = vm["zone_r2"]
    rotation_zone_2.center = [-0.125, -0.125, 0.0055]
    rotation_zone_2.axis = [0, 0, -1]

    rotation_zone_3 = vm["zone_r3"]
    rotation_zone_3.center = [0.125, -0.125, 0.0055]
    rotation_zone_3.axis = [0, 0, 1]

    rotation_zone_4 = vm["zone_r4"]
    rotation_zone_4.center = [0.125, 0.125, 0.0055]
    rotation_zone_4.axis = [0, 0, -1]

    omega = 6000 * fl.u.rpm

[12:39:30] INFO: using: SI unit system for unit inference.

### Time Stepping

For this unsteady simulation, the time step size and total number of steps are determined based on a desired rotational increment per step and the total number of rotor revolutions to be simulated. We will start with 3° of rotation per time step for 5 revolutions.

In [4]:
with fl.SI_unit_system:
    # Time step size is calculated based on a specified rotation per time step.
    deg_per_time_step_0 = 3.0 * fl.u.deg
    time_step_0 = deg_per_time_step_0 / omega.to("deg/s")

    # Total number of steps is determined by the required number of revolutions.
    revolution_time_0 = 360 * fl.u.deg / omega.to("deg/s")
    steps_0 = int(5 * revolution_time_0 / time_step_0)

           INFO: using: SI unit system for unit inference.

### Simulation Parameters Object

The `SimulationParams` object gathers all settings for a simulation run, including reference geometry dimensions, operating conditions, solver settings, and boundary conditions. Previously specified rotation zones will be assigned to a `Rotation` model.

In [5]:
with fl.SI_unit_system:
    params = fl.SimulationParams(
        reference_geometry=fl.ReferenceGeometry(
            area=0.0447726728530549,
            moment_center=[0, 0, 0],
            moment_length=[0.11938, 0.11938, 0.11938],
        ),
        operating_condition=fl.AerospaceCondition.from_mach(
            mach=0,
            thermal_state=fl.ThermalState(temperature=293.15),
            reference_mach=0.21868415800906676,
        ),
        time_stepping=fl.Unsteady(
            step_size=time_step_0,
            steps=steps_0,
        ),
        models=[
            fl.Fluid(
                navier_stokes_solver=fl.NavierStokesSolver(
                    absolute_tolerance=1e-10,
                    relative_tolerance=0.1,
                    order_of_accuracy=1,
                    linear_solver=fl.LinearSolver(max_iterations=30),
                ),
                turbulence_model_solver=fl.SpalartAllmaras(
                    absolute_tolerance=1e-8,
                    relative_tolerance=0.1,
                    order_of_accuracy=1,
                    linear_solver=fl.LinearSolver(max_iterations=20),
                ),
            ),
            fl.Wall(
                surfaces=[
                    vm["zone_s/airframe"],
                    vm["zone_r1/blade1"],
                    vm["zone_r2/blade2"],
                    vm["zone_r3/blade3"],
                    vm["zone_r4/blade4"],
                ],
            ),
            fl.Freestream(surfaces=vm["zone_s/farfield"]),
            fl.Rotation(
                name="Rotation",
                volumes=[rotation_zone_1, rotation_zone_2, rotation_zone_3, rotation_zone_4],
                spec=fl.AngularVelocity(value=omega),
            ),
        ],
    )

           INFO: using: SI unit system for unit inference.

## 4. Executing the Simulation and Forking

The simulation is executed in stages. As can be seen based on the settings up above, our first run will be a coarse, first-order simulation in order to establish an initial flow field. Subsequent simulations are then 'forked' from previous ones. Forking a case initializes a new simulation from the results of an existing one, which can significantly reduce computational cost when iterating on simulation parameters. This is particularly useful for changing numerical schemes or introducing new outputs without restarting the simulation from scratch.

> Note: First order run is not required here, but is used for demonstration purposes nonetheless.

In [6]:
case = project.run_case(params, "First order run")

           INFO: using: SI unit system for unit inference.

[12:39:33] INFO: Successfully submitted:                                                                           
                   type   = Case                                                                                   
                   name   = First order run                                                                        
                   id     = case-f2793c8b-a0b4-4e34-b426-55ebc517d47b                                              
                   status = pending                                                                                
           

### Second Run with Higher Order Accuracy

For the second run, the order of accuracy for the solvers is increased to 2, and the time step is refined. The case is forked from the initial run to leverage the already computed flow field.

In [7]:
# For the second run, parameters are updated directly on the params object of the completed case.
# The order of accuracy is increased and the time step size is refined.
case.params.models[0].navier_stokes_solver.order_of_accuracy = 2
case.params.models[0].navier_stokes_solver.linear_solver = fl.LinearSolver(max_iterations=25)

case.params.models[0].turbulence_model_solver.order_of_accuracy = 2
case.params.models[0].turbulence_model_solver.linear_solver = fl.LinearSolver(max_iterations=25)

with fl.SI_unit_system:
    deg_per_time_step_1 = 0.404496 * fl.u.deg
    time_step_1 = deg_per_time_step_1 / omega.to("deg/s")

    revolution_time_1 = 360 * fl.u.deg / omega.to("deg/s")
    steps_1 = int(5 * revolution_time_1 / time_step_1)

    case.params.time_stepping.step_size = time_step_1
    case.params.time_stepping.steps = steps_1

case_fork_1 = project.run_case(case.params, "Second order run", fork_from=case)

[12:39:35] INFO: using: SI unit system for unit inference.

           INFO: using: SI unit system for unit inference.

           INFO: using: SI unit system for unit inference.

[12:39:37] INFO: Successfully submitted:                                                                           
                   type   = Case                                                                                   
                   name   = Second order run                                                                       
                   id     = case-fdeffbcb-388d-497e-95b3-60d7fedf131b                                              
                   status = pending                                                                                
           

### Final Run with Aeroacoustic Outputs

The final simulation is forked from the second-order run. `AeroAcousticOutput` is added to the simulation parameters to collect acoustic data from specified observer points.

In [8]:
case_fork_1.params.outputs = [
    fl.AeroAcousticOutput(
        observers=[
            fl.Observer(position=[0, -1.905, 0] * fl.u.m, group_name="1"),
            fl.Observer(position=[0, -1.7599905, -0.72901194] * fl.u.m, group_name="1"),
            fl.Observer(position=[0, -1.3470384, -1.3470384] * fl.u.m, group_name="1"),
            fl.Observer(position=[0.9525, -0.9525, -1.3470384] * fl.u.m, group_name="1"),
            fl.Observer(position=[1.3470384, 0, -1.3470384] * fl.u.m, group_name="1"),
            fl.Observer(position=[0, 0, 1.905] * fl.u.m, group_name="1"),
            fl.Observer(position=[0, -0.37164706, 1.8683959] * fl.u.m, group_name="1"),
            fl.Observer(position=[0, -1.868396, 0.37164707] * fl.u.m, group_name="1"),
            fl.Observer(position=[1.295, 0, -0.767] * fl.u.m, group_name="2"),
        ],
        write_per_surface_output=True,
    )
]

case_fork_2 = project.run_case(case_fork_1.params, "Final run", fork_from=case_fork_1)

[12:39:41] INFO: using: SI unit system for unit inference.

           INFO: using: SI unit system for unit inference.

[12:39:43] INFO: Successfully submitted:                                                                           
                   type   = Case                                                                                   
                   name   = Final run                                                                              
                   id     = case-810a9648-9d02-4d60-b174-04bb918531ea                                              
                   status = pending                                                                                
           

## 5. Retrieving Results

After launching the simulations, the script waits for the final case to complete. The results, including the aeroacoustic data, can then be accessed and downloaded for analysis.

In [9]:
case_fork_2.wait()

results = case_fork_2.results

total_acoustics = results.aeroacoustics
print(total_acoustics)

[16:23:32] INFO: Saved to                                                                                          
           /var/folders/qk/mywsrvps5gl_f3yjx2k1v1xm0000gn/T/tmp4flfa4ln/fe98dd36-562e-4528-9753-c04ea964ab7a.csv

           time  physical_step  observer_0_pressure  observer_1_pressure  \
0     21.596813           5600         5.720588e-07        -1.969728e-07   
1     21.600670           5601         5.362895e-07        -2.111919e-07   
2     21.604527           5602         5.343675e-07        -2.184311e-07   
3     21.608383           5603         5.279445e-07        -2.306726e-07   
4     21.612240           5604         5.146239e-07        -2.314757e-07   
...         ...            ...                  ...                  ...   
4236  37.933262           9836        -1.430596e-06        -1.404098e-06   
4237  37.937119           9837        -1.393013e-06        -1.364033e-06   
4238  37.940975           9838        -1.345864e-06        -1.335681e-06   
4239  37.944832           9839        -1.302765e-06        -1.301096e-06   
4240  37.948689           9840        -1.236518e-06        -1.253773e-06   

      observer_2_pressure  observer_3_pressure  observer_4_pressure  \
0               

### Surface-Specific Outputs

In addition to the total aeroacoustic output from all surfaces, individual surface contributions can be downloaded. The following cell downloads the acoustic data for each of the four rotors and loads them into pandas DataFrames.

In [10]:
# There are also surface specific aeroacoustic output files
blade_1_acoustics = results.download_file_by_name(
    "results/surface_zone_r1_blade1_acoustics_v3.csv", to_folder="aeroacoustic_results"
)
blade_1_acoustics = pd.read_csv(blade_1_acoustics)
print("Blade 1 acoustics:")
print(blade_1_acoustics)

blade_2_acoustics = results.download_file_by_name(
    "results/surface_zone_r2_blade2_acoustics_v3.csv", to_folder="aeroacoustic_results"
)
blade_2_acoustics = pd.read_csv(blade_2_acoustics)
print("\nBlade 2 acoustics:")
print(blade_2_acoustics)

blade_3_acoustics = results.download_file_by_name(
    "results/surface_zone_r3_blade3_acoustics_v3.csv", to_folder="aeroacoustic_results"
)
blade_3_acoustics = pd.read_csv(blade_3_acoustics)
print("\nBlade 3 acoustics:")
print(blade_3_acoustics)

blade_4_acoustics = results.download_file_by_name(
    "results/surface_zone_r4_blade4_acoustics_v3.csv", to_folder="aeroacoustic_results"
)
blade_4_acoustics = pd.read_csv(blade_4_acoustics)
print("\nBlade 4 acoustics:")
print(blade_4_acoustics)

[16:23:41] INFO: Saved to aeroacoustic_results/surface_zone_r1_blade1_acoustics_v3.csv

Blade 1 acoustics:
           time   physical_step   observer_0_pressure   observer_1_pressure  \
0     21.596813            5600          1.645042e-07         -2.097362e-08   
1     21.600670            5601          1.522135e-07         -2.513679e-08   
2     21.604527            5602          1.568220e-07         -2.604763e-08   
3     21.608383            5603          1.534737e-07         -3.177862e-08   
4     21.612240            5604          1.497509e-07         -3.112281e-08   
...         ...             ...                   ...                   ...   
4294  38.156944            9894         -4.839658e-08         -1.803780e-07   
4295  38.160801            9895         -4.612721e-08         -1.767865e-07   
4296  38.164658            9896         -4.464747e-08         -1.787573e-07   
4297  38.168514            9897         -4.381082e-08         -1.792401e-07   
4298  38.172367            9898         -3.970325e-08         -1.790867e-07   

       observer_2_pressure   obs

[16:23:44] INFO: Saved to aeroacoustic_results/surface_zone_r2_blade2_acoustics_v3.csv


Blade 2 acoustics:
           time   physical_step   observer_0_pressure   observer_1_pressure  \
0     21.542822            5586          1.575615e-07         -8.122591e-08   
1     21.546679            5587          1.550869e-07         -8.154808e-08   
2     21.550535            5588          1.532802e-07         -8.497518e-08   
3     21.554392            5589          1.476895e-07         -8.319915e-08   
4     21.558249            5590          1.457267e-07         -8.265117e-08   
...         ...             ...                   ...                   ...   
4308  38.156944            9894          4.361937e-08         -2.392730e-07   
4309  38.160801            9895          4.591889e-08         -2.351592e-07   
4310  38.164658            9896          4.929480e-08         -2.321950e-07   
4311  38.168514            9897          5.211020e-08         -2.248036e-07   
4312  38.172367            9898          5.739892e-08         -2.251031e-07   

       observer_2_pressure   ob

[16:23:47] INFO: Saved to aeroacoustic_results/surface_zone_r3_blade3_acoustics_v3.csv


Blade 3 acoustics:
           time   physical_step   observer_0_pressure   observer_1_pressure  \
0     21.403986            5550          2.192105e-07         -1.645585e-07   
1     21.407843            5551          2.162504e-07         -1.563119e-07   
2     21.411699            5552          2.180365e-07         -1.511738e-07   
3     21.415556            5553          2.132866e-07         -1.449185e-07   
4     21.419413            5554          2.182970e-07         -1.412974e-07   
...         ...             ...                   ...                   ...   
4286  37.933262            9836         -1.299133e-07         -3.628256e-07   
4287  37.937119            9837         -1.294328e-07         -3.577859e-07   
4288  37.940975            9838         -1.270968e-07         -3.564439e-07   
4289  37.944832            9839         -1.253298e-07         -3.582451e-07   
4290  37.948689            9840         -1.203186e-07         -3.571033e-07   

       observer_2_pressure   ob

[16:23:50] INFO: Saved to aeroacoustic_results/surface_zone_r4_blade4_acoustics_v3.csv


Blade 4 acoustics:
           time   physical_step   observer_0_pressure   observer_1_pressure  \
0     21.596813            5600          1.633495e-07         -4.905565e-08   
1     21.600670            5601          1.520510e-07         -5.218257e-08   
2     21.604527            5602          1.505213e-07         -5.796136e-08   
3     21.608383            5603          1.518591e-07         -6.289314e-08   
4     21.612240            5604          1.480509e-07         -6.798292e-08   
...         ...             ...                   ...                   ...   
4236  37.933262            9836         -1.381269e-07         -1.384760e-07   
4237  37.937119            9837         -1.379311e-07         -1.395333e-07   
4238  37.940975            9838         -1.331161e-07         -1.475768e-07   
4239  37.944832            9839         -1.348998e-07         -1.535266e-07   
4240  37.948689            9840         -1.296906e-07         -1.596224e-07   

       observer_2_pressure   ob